In [127]:
import os
import pickle
import scipy 
import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.signal import get_window
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
from sklearn.decomposition import MiniBatchDictionaryLearning

## Feartures Extraction

In [128]:
# def extract_features(file_path):

#     x, sr = librosa.load(file_path)

#     freqs = np.fft.fftfreq(x.size)

#     def describe_freq(freqs):
#         mean = np.mean(freqs)
#         std = np.std(freqs) 
#         maxv = np.amax(freqs) 
#         minv = np.amin(freqs) 
#         median = np.median(freqs)
#         skew = scipy.stats.skew(freqs)
#         kurt = scipy.stats.kurtosis(freqs)
#         q1 = np.quantile(freqs, 0.25)
#         q3 = np.quantile(freqs, 0.75)
#         mode = scipy.stats.mode(freqs)[0][0]
#         iqr = scipy.stats.iqr(freqs)

#         return [mean, std, maxv, minv, median, skew, kurt, q1, q3, mode, iqr]

#     def rmse(x):
#         return [np.sum(x**2)]

#     def rmse2(x):
#         return [np.sqrt(np.mean(x**2))]

#     zero_crossings = [sum(librosa.zero_crossings(x, pad=False))]

#     tempo = [librosa.beat.tempo(x)[0]]

#     # mfcc=librosa.feature.mfcc(x)
#     mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=1)



#     hop_length = 512
#     oenv      = librosa.onset.onset_strength(y=x, sr=sr, hop_length=hop_length)
#     # tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)
#     spec_centroid = librosa.feature.spectral_centroid(x)[0]
#     spectral_bandwidth=librosa.feature.spectral_bandwidth(x)[0]
#     spectral_contrast=librosa.feature.spectral_contrast(x)[0]
#     spectral_flatness=librosa.feature.spectral_flatness(x)[0]
#     spectral_rolloff=librosa.feature.spectral_rolloff(x)[0]

#     combined = np.hstack([describe_freq(freqs),rmse(x),rmse2(x),zero_crossings,tempo,mfcc[0],oenv,spec_centroid,spectral_bandwidth,
#                         spectral_contrast, spectral_flatness, spectral_rolloff]) 
#     return combined

# extract_features('./files/magdy/100.wav')

##### MFCC Feature

In [129]:
def calculate_delta(array):

    rows,cols = array.shape
    print(rows)
    print(cols)
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first =0
            else:
                first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(file_path):
    audio , sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfcc_feature = mfcc.mfcc(audio,sample_rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
    print(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    
    combined = np.hstack((mfcc_feature,delta))
    
    return combined

In [130]:
names = ['OTD','CTD','OTW','CTW']
features = {}
samples = {}
# directory = './Speaker_Train_Data/'
# for audio in os.listdir(directory):
#     audio_path = directory + audio
#     if 'Michael'

# featuresMichael = []
# featuresAhmed = []
# featuresBelal = []
# featuresOthers = []
print(os.getcwd())
directory = './Speaker_Train_Data/'
for audio in os.listdir(directory):
    audio_path = directory + audio
    noNames = True
    for name in names:
        if name in audio:
            noNames = False
            if name not in features:
                features[name] = []
                samples[name] = 0
            features[name] = np.vstack(extract_features(audio_path) )
            samples[name] += 1
    if noNames:
        if 'others' not in features:
            features['others'] = []
            samples['others'] = 0
        features['others'] = np.vstack(extract_features(audio_path))
        samples['others'] += 1
    print(samples)

d:\SBME\3rd year\DSP\repo final\audio-recognition-2\Ahmed_Belal_Micheal
[[-1.73623492 -0.50451577  1.93144308 ...  2.67112852  0.37860671
   0.01531219]
 [-0.94921178  0.12338125  1.48488123 ...  1.71902809  1.40020765
   0.53952189]
 [-0.90534863  0.11268072  1.51496681 ...  1.54459365  1.78517215
   0.72090379]
 ...
 [ 0.11314614 -0.11822816 -0.93656715 ...  1.24985524 -0.30926458
  -0.14593034]
 [ 0.22226859 -0.10423302 -1.0165006  ...  0.48672734 -1.15411973
  -0.46771513]
 [ 0.19594677 -0.10082724 -1.2645474  ...  0.72614696 -0.41650277
  -0.36635967]]
198
20


C:\Users\somakiko\AppData\Local\Temp\ipykernel_4564\1728697229.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentenceFeatures = np.array([chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr])


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 2 has 1 dimension(s)

In [ ]:
models = {}
for name,val in features.items():
    models[name] = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
    models[name].fit(val)

# michael_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
# michael_gmm.fit(featuresMichael)

# belal_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
# belal_gmm.fit(featuresBelal)

# ahmed_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
# ahmed_gmm.fit(featuresAhmed)

# others_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
# others_gmm.fit(featuresOthers)

# open_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
# open_gmm.fit(openFeatures)

In [ ]:
# # Initialise the subplot function using number of rows and columns
# figure, axis = plt.subplots(2, 2)

# # For Sine Function
# axis[0, 0].plot(features['Michael'],'ro')
# axis[0, 0].set_title("michael")

# # For Cosine Function
# axis[0, 1].plot(features['Belal'])
# axis[0, 1].set_title("Belal")

# # For Tangent Function
# axis[1, 0].plot(features['ahmed_ashraf'])
# axis[1, 0].set_title("ahmed_ashraf")

# # For Tanh Function
# axis[1, 1].plot(features['others'])
# axis[1, 1].set_title("others")

# names = ['group_a', 'group_b', 'group_c']
# values = [1, 10, 100]

# # Combine all the operations and display
# plt.show()

In [ ]:
for key,val in models.items():
    pickle.dump(val,open(key+'.gmm','wb'))
# pickle.dump(michael_gmm,open('mayar.gmm','wb'))
# pickle.dump(belal_gmm,open('mina.gmm','wb'))
# pickle.dump(ahmed_gmm,open('magdy.gmm','wb'))
# pickle.dump(others_gmm,open('mostafa.gmm','wb'))
# pickle.dump(open_gmm,open('open.gmm','wb'))

In [ ]:
test = extract_features('./testing_set/sample.wav')
print(test.size)

[[-2.12283146e+00 -3.11029332e+00 -2.45874201e-02 ...  1.25697762e+00
   8.79253506e-02 -7.26827838e-01]
 [-2.10684288e+00 -3.06846582e+00 -2.58497052e-02 ...  1.82507820e-01
   6.11102389e-01 -3.60049720e-01]
 [-2.12182750e+00 -3.25984272e+00 -4.99748252e-01 ... -7.97423266e-01
   5.04621724e-01  3.42559153e-02]
 ...
 [-1.02747483e+00 -5.12916721e-01  1.32106030e+00 ... -2.59034201e-01
   5.48249644e-01  1.32213086e+00]
 [-8.87451273e-01 -1.35024487e-01  1.92176272e+00 ... -2.20977113e-01
   1.05990225e+00  8.35080524e-01]
 [-8.92624005e-01  3.06256969e-03  2.09456893e+00 ... -4.74581214e-01
   6.25063010e-01  1.14822987e+00]]
198
20
6


C:\Users\somakiko\AppData\Local\Temp\ipykernel_4564\1574464318.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentenceFeatures = np.array([chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr])


In [ ]:
for modelName,model in models.items():
    print(f'{modelName} scored {model.score(test)}')
# scores_1=np.array(open_gmm.score(test))
# scores_2=np.array(michael_gmm.score(test))
# scores_3=np.array(belal_gmm.score(test))
# scores_4=np.array(ahmed_gmm.score(test))
# scores_5=np.array(others_gmm.score(test))

# print(scores_1)
# print(scores_2)
# print(scores_3)
# print(scores_4)
# print(scores_5)


ValueError: setting an array element with a sequence.